# SO-ARM 数据采集、训练和评估完整流程

本教程整合了数据采集、模型训练和评估的完整流程，基于 Seeed Studio 官方指南。

**前置条件：**
- ✅ 已完成机械臂校准（01_calibration_and_teleoperation.ipynb）
- ✅ 遥操作功能正常
- ✅ 摄像头已配置

**本教程包含：**
1. 数据采集（录制演示）
2. 数据可视化和验证
3. 数据回放测试
4. 模型训练（ACT/SmolVLA/Pi0）
5. 模型评估和部署

**参考：** [Seeed Studio LeRobot Wiki](https://wiki.seeedstudio.com/cn/lerobot_so100m_new/)

---
## 📊 第一部分：数据采集

### 什么是数据采集？

**数据采集**是通过遥操作录制机械臂执行任务的过程：
- 使用 Leader Arm 演示任务
- Follower Arm 跟随并录制
- 同时记录摄像头画面
- 保存为训练数据集

### 数据质量的重要性

**高质量数据 = 高性能模型**

- 演示要流畅自然
- 包含完整的任务流程
- 保持摄像头视角固定
- 环境光照稳定
- 物体在画面中清晰可见

---
## 🎬 步骤 1: 准备数据采集

### 1.1 定义任务

首先明确您要教机械臂执行什么任务：

**示例任务：**
- 抓取方块并放置到目标位置
- 拿起笔写字
- 分类物体
- 组装零件

**任务设计原则：**
1. **简单明确** - 从单一动作开始
2. **可重复** - 物体位置可以变化但任务逻辑相同
3. **可视化** - 摄像头能看到关键信息
4. **成功可判断** - 有明确的完成标志

### 1.2 环境设置

**工作空间准备：**
- 清理桌面，移除无关物体
- 固定摄像头位置（使用三脚架或支架）
- 确保光照充足且均匀
- 标记物体的起始和目标位置

**摄像头设置：**
- 分辨率：640x480（推荐）
- 帧率：30 fps
- 视角：能看到完整任务区域
- 对焦：清晰对焦到工作区域

### 1.3 配置数据采集参数

In [ ]:
# ========== 数据集配置 ==========

# Hugging Face 配置
HF_USER = "your_username"  # 修改为您的 HF 用户名
DATASET_NAME = "so101_pick_place"  # 数据集名称
REPO_ID = f"{HF_USER}/{DATASET_NAME}"

# 任务描述（英文，用于语言条件模型）
TASK_DESCRIPTION = "Pick up the red cube and place it on the green plate"

# 采集参数
NUM_EPISODES = 50  # 推荐至少 50 个
EPISODE_TIME_S = 30  # 每个 episode 最大时长（秒）

# 是否上传到 Hub
PUSH_TO_HUB = False  # 先本地保存，验证后再上传

# 机械臂配置（从校准步骤获得）
ROBOT_PORT = "/dev/ttyACM0"
ROBOT_ID = "my_so101_follower"
TELEOP_PORT = "/dev/ttyACM1"
TELEOP_ID = "my_so101_leader"

# 摄像头配置
CAMERA_INDEX = 0  # 主摄像头索引

print("✅ 数据采集配置：")
print(f"  数据集: {REPO_ID}")
print(f"  任务: {TASK_DESCRIPTION}")
print(f"  Episodes: {NUM_EPISODES}")
print(f"  上传 Hub: {PUSH_TO_HUB}")

---
## 🎥 步骤 2: 开始录制数据

### 2.1 基础录制命令（本地保存）

In [ ]:
print("在终端运行以下命令开始录制：")
print()
print("lerobot-record \\")
print(f"  --robot.type=so101_follower \\")
print(f"  --robot.port={ROBOT_PORT} \\")
print(f"  --robot.id={ROBOT_ID} \\")
print(f"  --robot.cameras='{{front: {{type: opencv, index_or_path: {CAMERA_INDEX}, \\")
print(f"    width: 640, height: 480, fps: 30, fourcc: \"MJPG\"}}}}' \\")
print(f"  --teleop.type=so101_leader \\")
print(f"  --teleop.port={TELEOP_PORT} \\")
print(f"  --teleop.id={TELEOP_ID} \\")
print(f"  --dataset.repo_id={REPO_ID} \\")
print(f"  --dataset.single_task=\"{TASK_DESCRIPTION}\" \\")
print(f"  --dataset.num_episodes={NUM_EPISODES} \\")
print(f"  --dataset.episode_time_s={EPISODE_TIME_S} \\")
print(f"  --dataset.push_to_hub={str(PUSH_TO_HUB).lower()}")
print()
print("⚠️ 重要提示：")
print("  - 录制开始后，Leader Arm 进入录制模式")
print("  - 操作 Leader Arm 演示任务")
print("  - 完成后按提示键结束当前 episode")
print("  - 重复直到录制完所有 episodes")

### 2.2 录制技巧和最佳实践

#### 录制前
1. **练习几次** - 熟悉动作流程
2. **确认视角** - 检查摄像头画面
3. **准备物体** - 多个物体便于连续录制
4. **测试一个 episode** - 确保系统正常

#### 录制中
1. **动作流畅** - 避免突然加速或停顿
2. **速度适中** - 不要太快或太慢
3. **完成任务** - 每个 episode 包含完整流程
4. **变化位置** - 改变物体起始位置增加多样性

#### 推荐数据分布
- **50 episodes** = 5 个位置 × 每个位置 10 次
- 起始位置 A：10 个成功演示
- 起始位置 B：10 个成功演示  
- 起始位置 C：10 个成功演示
- 起始位置 D：10 个成功演示
- 起始位置 E：10 个成功演示

#### 避免的错误
- ❌ 失败的演示（掉落物体、碰撞）
- ❌ 不完整的动作
- ❌ 摄像头移动或遮挡
- ❌ 复杂背景或人员走动
- ❌ 光照突然变化

---
## 👁️ 步骤 3: 数据可视化

### 3.1 加载数据集

In [ ]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset

print("📂 加载数据集...")

# 从本地加载
dataset = LeRobotDataset(
    repo_id=REPO_ID,
    root=None,  # 使用默认路径
)

print(f"✅ 数据集加载成功")
print(f"  Episodes: {dataset.num_episodes}")
print(f"  Frames: {dataset.num_frames}")
print(f"  FPS: {dataset.fps}")
print(f"  特征: {list(dataset.features.keys())}")

### 3.2 可视化第一帧

In [ ]:
import matplotlib.pyplot as plt

# 获取第一帧
frame = dataset[0]

# 显示图像
image_keys = [k for k in frame.keys() if 'image' in k]

if image_keys:
    fig, axes = plt.subplots(1, len(image_keys), figsize=(12, 4))
    if len(image_keys) == 1:
        axes = [axes]
    
    for i, key in enumerate(image_keys):
        img = frame[key].numpy()
        axes[i].imshow(img)
        axes[i].set_title(key)
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# 显示状态和动作
if 'observation.state' in frame:
    print(f"\n关节状态: {frame['observation.state'].numpy()}")
if 'action' in frame:
    print(f"动作: {frame['action'].numpy()}")

### 3.3 统计信息

In [ ]:
import numpy as np

# Episode 长度统计
episode_lengths = []
for ep in range(dataset.num_episodes):
    start = dataset.episode_data_index["from"][ep]
    end = dataset.episode_data_index["to"][ep]
    episode_lengths.append(end - start)

print("📊 数据集统计：")
print(f"\nEpisode 帧数：")
print(f"  平均: {np.mean(episode_lengths):.1f} 帧")
print(f"  最小: {np.min(episode_lengths)} 帧")
print(f"  最大: {np.max(episode_lengths)} 帧")

# 转换为时间
fps = dataset.fps
print(f"\nEpisode 时长 (@ {fps} FPS)：")
print(f"  平均: {np.mean(episode_lengths)/fps:.1f} 秒")
print(f"  最小: {np.min(episode_lengths)/fps:.1f} 秒")
print(f"  最大: {np.max(episode_lengths)/fps:.1f} 秒")

# 绘制分布
plt.figure(figsize=(10, 5))
plt.hist(episode_lengths, bins=20, edgecolor='black')
plt.xlabel('帧数')
plt.ylabel('Episode 数量')
plt.title('Episode 长度分布')
plt.grid(True, alpha=0.3)
plt.show()

---
## 🔄 步骤 4: 数据回放测试

**数据回放**可以验证录制的数据是否正确。

In [ ]:
# 选择要回放的 episode
REPLAY_EPISODE = 0  # 回放第一个 episode

print("在终端运行以下命令回放数据：")
print()
print("lerobot-replay \\")
print(f"  --robot.type=so101_follower \\")
print(f"  --robot.port={ROBOT_PORT} \\")
print(f"  --robot.id={ROBOT_ID} \\")
print(f"  --dataset.repo_id={REPO_ID} \\")
print(f"  --dataset.episode={REPLAY_EPISODE}")
print()
print("预期行为：")
print("  - Follower Arm 会重现录制的动作")
print("  - 轨迹应该与演示时相似")
print("  - 如果偏差大，可能需要重新录制")

---
## 🧠 第二部分：模型训练

### 支持的策略

LeRobot 支持多种策略，各有特点：

| 策略 | 特点 | 推荐场景 |
|------|------|----------|
| **ACT** | 经典方法，稳定可靠 | 通用任务，初学者 |
| **SmolVLA** | 视觉语言模型，泛化好 | 语言条件任务 |
| **Pi0** | 新一代策略，性能强 | 高精度要求 |

### 训练硬件要求

- **GPU**: 推荐 A100 / RTX 3090 / RTX 4090
- **显存**: 至少 16GB
- **RAM**: 32GB+
- **存储**: 100GB+ 用于 checkpoints

---
## 🚀 步骤 5: 训练 ACT 策略

In [ ]:
# ACT 训练配置
ACT_OUTPUT_DIR = "outputs/train/act_so101"
ACT_STEPS = 300000  # 推荐 300k steps
ACT_BATCH_SIZE = 32  # 根据 GPU 调整

print("在终端运行以下命令训练 ACT：")
print()
print("lerobot-train \\")
print(f"  --dataset.repo_id={REPO_ID} \\")
print(f"  --policy.type=act \\")
print(f"  --output_dir={ACT_OUTPUT_DIR} \\")
print(f"  --steps={ACT_STEPS} \\")
print(f"  --batch_size={ACT_BATCH_SIZE}")
print()
print("训练时间估算：")
print("  - A100: ~6-8 小时")
print("  - RTX 3090: ~10-12 小时")
print()
print("💡 使用 tmux 或 screen 在后台运行，避免中断！")

---
## 🌟 步骤 6: 训练 SmolVLA 策略

In [ ]:
# SmolVLA 训练配置
SMOLVLA_OUTPUT_DIR = "outputs/train/smolvla_so101"
SMOLVLA_STEPS = 20000  # SmolVLA 微调步数较少
SMOLVLA_BATCH_SIZE = 64

print("在终端运行以下命令训练 SmolVLA：")
print()
print("lerobot-train \\")
print(f"  --policy.path=lerobot/smolvla_base \\")
print(f"  --dataset.repo_id={REPO_ID} \\")
print(f"  --output_dir={SMOLVLA_OUTPUT_DIR} \\")
print(f"  --steps={SMOLVLA_STEPS} \\")
print(f"  --batch_size={SMOLVLA_BATCH_SIZE}")
print()
print("训练时间估算：")
print("  - A100: ~2-3 小时")
print("  - RTX 3090: ~4-5 小时")
print()
print("⚠️ SmolVLA 需要先安装：pip install -e '.[smolvla]'")

---
## 🔬 步骤 7: 训练 Pi0 策略

In [ ]:
# Pi0 训练配置
PI0_OUTPUT_DIR = "outputs/train/pi0_so101"
PI0_STEPS = 20000
PI0_BATCH_SIZE = 32

print("在终端运行以下命令训练 Pi0：")
print()
print("lerobot-train \\")
print(f"  --policy.type=pi0 \\")
print(f"  --dataset.repo_id={REPO_ID} \\")
print(f"  --output_dir={PI0_OUTPUT_DIR} \\")
print(f"  --steps={PI0_STEPS} \\")
print(f"  --batch_size={PI0_BATCH_SIZE}")
print()
print("训练时间估算：")
print("  - A100: ~3-4 小时")
print("  - RTX 3090: ~5-6 小时")

---
## 📊 步骤 8: 监控训练

### 8.1 查看训练日志

In [ ]:
from pathlib import Path

# 选择要检查的输出目录
output_dir = Path(ACT_OUTPUT_DIR)

if output_dir.exists():
    # 查找 checkpoints
    checkpoints = sorted(output_dir.glob("checkpoints/*"))
    
    if checkpoints:
        print(f"📦 找到 {len(checkpoints)} 个 checkpoints:\n")
        for cp in checkpoints:
            print(f"  - {cp.name}")
        print(f"\n✅ 最新: {checkpoints[-1].name}")
    else:
        print("⚠️ 还没有 checkpoints")
        print("训练可能刚开始")
else:
    print("⚠️ 输出目录不存在")
    print(f"请确认训练已开始: {output_dir}")

---
## 🎯 第三部分：模型评估和部署

### 什么是评估？

**评估 (Evaluation)** 是测试训练好的模型在实际机械臂上的表现：
- 加载训练好的 checkpoint
- 让机械臂自主执行任务
- 统计成功率和性能

**与训练数据回放的区别：**
- 回放：重现录制的动作（已知正确）
- 评估：模型自主决策（测试学习效果）

---
## 🧪 步骤 9: 评估模型

In [ ]:
# 评估配置
MODEL_PATH = f"{ACT_OUTPUT_DIR}/checkpoints/last/pretrained_model"
EVAL_DATASET = f"{HF_USER}/eval_{DATASET_NAME}"  # 评估结果保存位置
EVAL_EPISODES = 10  # 评估次数

print("在终端运行以下命令评估模型：")
print()
print("lerobot-record \\")
print(f"  --robot.type=so101_follower \\")
print(f"  --robot.port={ROBOT_PORT} \\")
print(f"  --robot.id={ROBOT_ID} \\")
print(f"  --robot.cameras='{{front: {{type: opencv, index_or_path: {CAMERA_INDEX}, \\")
print(f"    width: 640, height: 480, fps: 30, fourcc: \"MJPG\"}}}}' \\")
print(f"  --policy.path={MODEL_PATH} \\")
print(f"  --dataset.repo_id={EVAL_DATASET} \\")
print(f"  --dataset.num_episodes={EVAL_EPISODES} \\")
print(f"  --dataset.push_to_hub=false")
print()
print("⚠️ 重要：")
print("  - 机械臂会自主运动，确保安全！")
print("  - 准备好随时按紧急停止")
print("  - 每个 episode 后手动重置物体位置")
print("  - 记录成功和失败的次数")

---
## 📈 步骤 10: 分析评估结果

In [ ]:
# 手动记录评估结果
print("评估记录表：")
print()
print("| Episode | 成功? | 失败原因 | 备注 |")
print("|---------|-------|---------|------|")
for i in range(EVAL_EPISODES):
    print(f"| {i+1:2d}      |       |         |      |")
print()
print("成功率 = 成功次数 / 总次数")
print()
print("性能标准：")
print("  优秀: ≥ 90%")
print("  良好: 70-90%")
print("  一般: 50-70%")
print("  需改进: < 50%")

---
## 🔧 常见问题和解决方案

### 数据采集问题

**Q1: 录制时机械臂动作卡顿？**

A: 检查：
- USB 是否直连（不要用 Hub）
- 摄像头分辨率是否过高
- 系统资源是否充足

**Q2: 数据集保存在哪里？**

A: 默认保存在 `~/.cache/huggingface/lerobot/`

### 训练问题

**Q3: 训练显存不足（OOM）？**

A: 减小 batch_size：
- A100: 32-64
- RTX 3090: 16-32  
- RTX 3080: 8-16

**Q4: 训练多久合适？**

A: 观察 loss 曲线：
- Loss 持续下降：继续训练
- Loss 平稳：可以停止
- Loss 上升：过拟合，用早期 checkpoint

### 评估问题

**Q5: 评估时报错 "File exists"？**

A: 删除之前的评估数据：
```bash
rm -rf ~/.cache/huggingface/lerobot/eval_*
```

**Q6: 模型表现不好怎么办？**

A: 改进策略：
1. 采集更多数据（100+ episodes）
2. 提高数据质量（更流畅的演示）
3. 增加数据多样性（更多起始位置）
4. 延长训练时间
5. 尝试不同的策略

---
## ✅ 完成！

恭喜您完成了从数据采集到模型评估的完整流程！

### 总结

您已经学会了：
- ✅ 如何录制高质量的演示数据
- ✅ 数据可视化和验证方法
- ✅ 训练三种不同的策略（ACT/SmolVLA/Pi0）
- ✅ 评估和部署训练好的模型
- ✅ 故障排查和性能优化

### 下一步

根据评估结果：

**如果成功率 ≥ 70%：**
- 尝试更复杂的任务
- 探索多任务学习
- 分享您的模型和经验

**如果成功率 < 70%：**
- 分析失败案例
- 采集针对性数据
- 调整训练超参数
- 尝试不同策略

### 进阶主题

- 多任务学习
- 语言条件控制
- 迁移学习
- 在线学习和微调

---

**参考资源：**
- [Seeed Studio Wiki](https://wiki.seeedstudio.com/cn/lerobot_so100m_new/)
- [LeRobot 文档](https://huggingface.co/docs/lerobot)
- [社区 Discord](https://discord.com/invite/s3KuuzsPFb)

**🎉 祝您训练出优秀的机械臂策略！**